# Federated Learning with FedAdam for Cybersecurity Dataset

This notebook implements federated learning using FedAdam optimization for cybersecurity attack detection.


In [16]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import our federated learning modules
from federated_learning.models import CyberSecurityNet
from federated_learning.client import FedClient
from federated_learning.server import FedAdamServer
from federated_learning.utils import prepare_data, create_data_loaders

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")


Using device: cpu
PyTorch version: 2.7.1


## Data Preparation

Loading and preprocessing the majority and minority datasets for federated learning.


In [17]:
# Prepare data for both majority and minority datasets
print("Preparing majority dataset...")
X_train_maj, X_test_maj, y_train_maj, y_test_maj, scaler_maj, le_maj = prepare_data('output/majority.csv')

print("\nPreparing minority dataset...")
X_train_min, X_test_min, y_train_min, y_test_min, scaler_min, le_min = prepare_data('output/minority.csv')

print(f"\nDataset Information:")
print(f"Majority dataset - Train: {X_train_maj.shape}, Test: {X_test_maj.shape}")
print(f"Minority dataset - Train: {X_train_min.shape}, Test: {X_test_min.shape}")
print(f"Number of features: {X_train_maj.shape[1]}")
print(f"Majority classes: {len(np.unique(y_train_maj))}")
print(f"Minority classes: {len(np.unique(y_train_min))}")

# Display class distributions
print(f"\nMajority dataset class distribution:")
unique_maj, counts_maj = np.unique(y_train_maj, return_counts=True)
for cls, count in zip(unique_maj, counts_maj):
    print(f"  Class {cls} ({le_maj.classes_[cls]}): {count} samples")

print(f"\nMinority dataset class distribution:")
unique_min, counts_min = np.unique(y_train_min, return_counts=True)
for cls, count in zip(unique_min, counts_min):
    print(f"  Class {cls} ({le_min.classes_[cls]}): {count} samples")


Preparing majority dataset...

Preparing minority dataset...

Dataset Information:
Majority dataset - Train: (273688, 68), Test: (68423, 68)
Minority dataset - Train: (1422, 68), Test: (356, 68)
Number of features: 68
Majority classes: 4
Minority classes: 4

Majority dataset class distribution:
  Class 0 (0.0): 97553 samples
  Class 1 (1.0): 78503 samples
  Class 2 (2.0): 54739 samples
  Class 3 (3.0): 42893 samples

Minority dataset class distribution:
  Class 0 (4.0): 1124 samples
  Class 1 (5.0): 153 samples
  Class 2 (6.0): 131 samples
  Class 3 (7.0): 14 samples


In [18]:
# Create data loaders for each client
batch_size = 64

train_loader_maj, test_loader_maj = create_data_loaders(
    X_train_maj, y_train_maj, X_test_maj, y_test_maj, batch_size
)

train_loader_min, test_loader_min = create_data_loaders(
    X_train_min, y_train_min, X_test_min, y_test_min, batch_size
)

print("Data loaders created successfully!")
print(f"Majority client - Train batches: {len(train_loader_maj)}, Test batches: {len(test_loader_maj)}")
print(f"Minority client - Train batches: {len(train_loader_min)}, Test batches: {len(test_loader_min)}")


Data loaders created successfully!
Majority client - Train batches: 4277, Test batches: 1070
Minority client - Train batches: 23, Test batches: 6


## Model Setup

Setting up the neural network architecture and determining the number of classes.


In [19]:
# Model parameters
input_dim = X_train_maj.shape[1]
num_classes_maj = len(np.unique(y_train_maj))
num_classes_min = len(np.unique(y_train_min))

# For federated learning, we'll use the maximum number of classes
max_classes = max(num_classes_maj, num_classes_min)

print(f"Model Configuration:")
print(f"Input dimension: {input_dim}")
print(f"Number of classes (majority): {num_classes_maj}")
print(f"Number of classes (minority): {num_classes_min}")
print(f"Using max classes for model: {max_classes}")

# Display model architecture
sample_model = CyberSecurityNet(input_dim, max_classes)
print(f"\nModel Architecture:")
print(sample_model)

# Count parameters
total_params = sum(p.numel() for p in sample_model.parameters())
trainable_params = sum(p.numel() for p in sample_model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


Model Configuration:
Input dimension: 68
Number of classes (majority): 4
Number of classes (minority): 4
Using max classes for model: 4

Model Architecture:
CyberSecurityNet(
  (fc1): Linear(in_features=68, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=4, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

Total parameters: 19,300
Trainable parameters: 19,300


## Federated Learning Setup

Initializing the server and clients for federated learning with FedAdam optimization.


In [20]:
# Initialize global model
global_model = CyberSecurityNet(input_dim, max_classes)

# Initialize server with FedAdam
server = FedAdamServer(
    model=global_model,
    beta1=0.9,      # momentum parameter
    beta2=0.999,    # second moment parameter
    eta=0.01,       # server learning rate
    tau=1e-3        # regularization parameter
)

# Initialize clients
client_majority = FedClient(
    client_id="majority",
    model=CyberSecurityNet(input_dim, max_classes),
    train_loader=train_loader_maj,
    test_loader=test_loader_maj,
    device=device
)

client_minority = FedClient(
    client_id="minority",
    model=CyberSecurityNet(input_dim, max_classes),
    train_loader=train_loader_min,
    test_loader=test_loader_min,
    device=device
)

clients = [client_majority, client_minority]

print("Federated learning setup completed!")
print(f"Server: FedAdam with β1={server.beta1}, β2={server.beta2}, η={server.eta}")
print(f"Number of clients: {len(clients)}")
for client in clients:
    print(f"  - Client '{client.client_id}': {len(client.train_loader.dataset)} training samples")


Federated learning setup completed!
Server: FedAdam with β1=0.9, β2=0.999, η=0.01
Number of clients: 2
  - Client 'majority': 273688 training samples
  - Client 'minority': 1422 training samples


## Training Configuration

Setting up training parameters and metrics tracking.


In [21]:
# Training parameters
num_rounds = 50
local_epochs = 5
learning_rate = 0.001

# Tracking metrics
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

print(f"Training Configuration:")
print(f"Number of communication rounds: {num_rounds}")
print(f"Local epochs per round: {local_epochs}")
print(f"Client learning rate: {learning_rate}")
print(f"Batch size: {batch_size}")
print("\n" + "="*60)
print("Starting Federated Training...")
print("="*60)


Training Configuration:
Number of communication rounds: 50
Local epochs per round: 5
Client learning rate: 0.001
Batch size: 64

Starting Federated Training...


In [22]:
import time

start_time = time.time()

for round_num in range(num_rounds):
    round_start_time = time.time()
    print(f"\n🔄 Round {round_num + 1}/{num_rounds}")
    
    # Get global model parameters
    global_params = server.get_parameters()
    
    # Client training
    client_results = []
    
    for client in clients:
        # Set global parameters to client
        client.set_parameters(global_params)
        
        # Local training
        loss, num_samples = client.train(epochs=local_epochs, lr=learning_rate)
        
        # Get updated parameters
        updated_params = client.get_parameters()
        
        client_results.append((updated_params, num_samples, loss))
        print(f"  📱 Client {client.client_id}: Loss = {loss:.4f}, Samples = {num_samples}")
    
    # Server aggregation with FedAdam
    server.aggregate_fit(client_results)
    
    # Evaluation every 5 rounds or first round
    if (round_num + 1) % 5 == 0 or round_num == 0:
        eval_results = []
        
        # Update clients with new global model
        global_params = server.get_parameters()
        
        for client in clients:
            client.set_parameters(global_params)
            test_loss, test_acc = client.evaluate()
            eval_results.append((test_loss, test_acc, len(client.test_loader.dataset)))
            print(f"  📊 Client {client.client_id} Test: Loss = {test_loss:.4f}, Accuracy = {test_acc:.4f}")
        
        # Aggregate evaluation results
        avg_test_loss, avg_test_acc = server.aggregate_evaluate(eval_results)
        
        test_losses.append(avg_test_loss)
        test_accuracies.append(avg_test_acc)
        
        print(f"  🌐 Global Test: Loss = {avg_test_loss:.4f}, Accuracy = {avg_test_acc:.4f}")
    
    round_time = time.time() - round_start_time
    print(f"  ⏱️  Round time: {round_time:.2f}s")

total_time = time.time() - start_time
print(f"\n✅ Federated training completed!")
print(f"⏱️  Total training time: {total_time:.2f}s ({total_time/60:.1f} minutes)")
print(f"🎯 Final Global Test Accuracy: {test_accuracies[-1]:.4f}")



🔄 Round 1/50
  📱 Client majority: Loss = 0.0206, Samples = 1368440
  📱 Client minority: Loss = 0.6360, Samples = 7110
  📊 Client majority Test: Loss = 1.3415, Accuracy = 0.5555
  📊 Client minority Test: Loss = 1.3794, Accuracy = 0.1236
  🌐 Global Test: Loss = 1.3417, Accuracy = 0.5532
  ⏱️  Round time: 29.86s

🔄 Round 2/50
  📱 Client majority: Loss = 0.0187, Samples = 1368440
  📱 Client minority: Loss = 0.6334, Samples = 7110
  ⏱️  Round time: 26.86s

🔄 Round 3/50
  📱 Client majority: Loss = 0.0169, Samples = 1368440
  📱 Client minority: Loss = 0.6063, Samples = 7110
  ⏱️  Round time: 26.72s

🔄 Round 4/50
  📱 Client majority: Loss = 0.0157, Samples = 1368440
  📱 Client minority: Loss = 0.6039, Samples = 7110
  ⏱️  Round time: 26.77s

🔄 Round 5/50
  📱 Client majority: Loss = 0.0151, Samples = 1368440
  📱 Client minority: Loss = 0.5900, Samples = 7110
  📊 Client majority Test: Loss = 0.9230, Accuracy = 0.9645
  📊 Client minority Test: Loss = 1.4941, Accuracy = 0.4410
  🌐 Global Test: Lo